# PARENT VESSEL RECONSTRUCTION

*Text below extracted from http://www.vmtk.org/tutorials/ParentVesselReconstruction.html* 

This tutorial illustrates the steps for the digital removal of a saccular cerebral aneurysm from its parent vasculature. The framework of the algorithm was originally proposed in Ford et al, *An objective approach to digital removal of saccular aneurysm: techniques and applications. BJR, 2009, ss55-61*. It was successively implemented in vmtk by Marina Piccinelli.

The algorithm relies on the definition of Voronoi Diagram and on its properties, particularly on the fact that given the model surface its Voronoi Diagram can be derived and vice versa.

<hr>
### Requirements

This tutorial is based on three Python scripts located in the vmtkApps directory in the source code (vmtk/vmtkApps/CerebralAneurysms/ParentVesselReconstructions).

The programs are written make use of VTK and vmtk, which have to be installed for the scripts to run properly. Only the first script requires input data from the user, while the second and third steps read datasets and files created and saved along the various steps.

The initial input files and those created successively will be put in the same directory and an ID will be used to handle the case at hand. Eventually only the directory path and the ID will be provided to the scripts together with high-level information about the case (i.e. terminal vs lateral) or option selections (i.e. smoothing on vs off). The scripts will directly read in the given directory the needed files whose names are predefined with the exception of the ID as prefix (e.g. id_model.vtp, id_voronoi.vtp, etc).

<hr>
### Description

The removal of the aneurysm sac is performed in three successive steps, each one implemented in a separate python script.

1. **patchandinterpolatecenterlines.py**: Identification of points along the artery centerlines that define the area of influence of the sac on the parent vasculature, removal of the portion of centerlines between these points and creation of a new interpolated parent artery centerline;

2. **clipvoronoidiagram.py**: Removal from the complete Voronoi Diagram of the surface model of the portions related to the aneurysm sac and the parent artery lying underneath it;

3. **paralleltransportvoronoidiagram.py**: Reconstruction of a new Voronoi Diagram for the portion of parent artery just removed; this operation is performed parallel transporting Voronoi points at one end of the empty region to the other extreme following the interpolated centerlines.

Given the high morphological variability of real cases, many auxiliary objects and files are created along the execution of the programs; they allow checking of the results and, in case, tuning of tolerances.

In [3]:
from vmtk import vmtkscripts
from vmtk import pypes
import os

In [4]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

In [17]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath  = '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/'
pythonBin = '/home/iagolessa/bin/python/'

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

# Define case type here
caseType ='terminal'
caseId    = 'case11'

In [10]:
# Reading surface and voronoi diagrm file
surfaceModel = vmtk_functions.readSurface(parentVesselDir+caseId+"/"+caseId+"_model.vtp")
vmtk_functions.viewSurface(surfaceModel)

Reading VTK XML surface file.
Quit renderer


In [11]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surfaceModel
# voronoiDiagram.SurfaceInputFileName = surfaceInputFile #casePath+"surfaces/surfaceRemeshed.vtp" 
# voronoiDiagram.IORead()
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
# voronoiDiagram.DelaunayTessellationOutputFileName = casePath+"surfaces/delaunayTesselation.vtp"
voronoiDiagram.VoronoiDiagramOutputFileName = parentVesselDir+caseId+"/"+caseId+"_voronoi.vtp"
voronoiDiagram.Execute()
voronoiDiagram.IOWrite()

NonManifold check.
Cleaning surface.
Triangulating surface.
Writing VTK XML surface file.


In [14]:
# Reading voronoi diagram
voronoiName = parentVesselDir+caseId+"/"+caseId+"_voronoi.vtp"
voronoiDiagram = vmtk_functions.readSurface(voronoiName)
print('Voronoi diagram read.')
print(voronoiName)
vmtk_functions.viewSurface(voronoiDiagram)

Reading VTK XML surface file.
Voronoi diagram read.
/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/parentVessel/case11/case11_voronoi.vtp
Quit renderer


In [15]:
# Final comparison between the extracted surface and the initial VOI image
myPipe = 'vmtksurfacereader -ifile '+surfaceName+' ' \
         '--pipe vmtkrenderer --pipe vmtksurfaceviewer -ifile '+voronoiName+' -array MaximumInscribedSphereRadius -colormap rainbow -display 0 ' \
         '--pipe vmtksurfaceviewer -i @vmtksurfacereader.o -opacity 0.3'
        
print(myPipe)
pypes.PypeRun(myPipe)

NameError: name 'surfaceName' is not defined

In [ ]:
# # Visualizing two surfaces
# render = vmtkscripts.vmtkRenderer()
# render.Execute()

# ## WITH BUG!!!!

# view1 = vmtkscripts.vmtkSurfaceViewer()
# view1.Surface = surfaceModel
# view1.vmtkRenderer = render.vmtkRenderer
# view1.Display = 0
# view1.Opacity = 0.3

# view2 = vmtkscripts.vmtkSurfaceViewer()
# view2.Surface = voronoiDiagram
# view2.vmtkRenderer = render.vmtkRenderer
# view1.Execute()
# view2.Execute()

<hr/>
# Centerline Selection

### *Lateral Configuration*

Two couples of centerlines have to be provided. They are extracted from the model from the input section to the aneurysm dome and the output - forward direction, **id_forwardcl.vtp** - and from the output section to the aneurysm dome and the input - backward direction, **id_backwardcl.vtp**. Particular care has to be put in the order of selection of the target points in the two couples of centerlines (see figures below). More in details:

* For the forward direction: the source point should be at the input section, the first target point should be placed on the aneurysm sac dome, while the following target points can be placed on one or more vessel output sections downstream the aneurysm;

* For the backward direction, any of output sections can be selected as source point, the first target point is still the aneurysm dome, and the second target point should be the model input section.

<img src="./img/lateral1.png" style="width:200px;"/>

<img src="./img/lateral2.png" style="width:200px;"/>

### *Terminal Configuration*

Three couples of centerlines have to be provided for terminal configurations: the physiologically correct parent vasculature centerlines (id_parentvessel.vtp) and two couples of centerlines travelling from one daughter artery to the aneurysm dome and the other daughter artery. For terminal aneurysms too some care should be put in the selection of source and target points. In the creation of the parent vessel centerlines (Figure A) the order of selection of daughter artery output sections, i.e. of target points, is not important per se, but the creation of the following centerline couples (id_dau1cl.vtp and id_dau2cl.vtp) should be coherent with this initial ordering. The artery where the first target point is placed will become the source point of the daughter 1 centerlines (Figure B), while the artery where the second target point is placed has to be the source point for the daughter 2 centerlines (Fig. C). Centerlines are computed with *vmtkcenterlines* and the order in placing seeds should be carefully followed.

<img src="./img/terminalCl.png" style="width:700px;"/>

In [16]:
def generateCenterline(surface,centerlineOutputName):
    """To compute centerlines and resampling for parent vessel reconstruction"""
    # Computing centerlines in forward direction
    centerlines = vmtkscripts.vmtkCenterlines()
    centerlines.Surface = surface
    centerlines.AppendEndPoints = 1
    centerlines.Execute()

    # Resampling
    print("Resampling")
    centerlineResampling = vmtkscripts.vmtkCenterlineResampling()
    centerlineResampling.Centerlines = centerlines.Centerlines
    centerlineResampling.Length = 0.1
    centerlineResampling.CenterlinesOutputFileName = centerlineOutputName
    centerlineResampling.Execute()
    centerlineResampling.IOWrite()
    print("Done.")

def viewCenterlineFromFile(centerlineFileName):
    centerlineViewer = vmtkscripts.vmtkCenterlineViewer()
    centerlineViewer.CenterlinesInputFileName = centerlineFileName
    centerlineViewer.IORead()
    centerlineViewer.Execute()

In [18]:
# Creating the centerlines
if caseType == "lateral":
    # Files identifiers for lateral aneurysm cases
    forwardFileName = caseId+"_forwardcl.vtp" 
    backwardFileName = caseId+"_backwardcl.vtp"
    
    print('Computing forward centerlines.\n')
    # Computing forward centerlines
    forwardCenterlineName = parentVesselDir+caseId+"/"+forwardFileName
    generateCenterline(surfaceModel,forwardCenterlineName)
#     viewCenterlineFromFile(forwardCenterlineName)
    
    print('Computing backward centerlines.\n')
    # Computing backward centerlines
    backwardCenterlineName = parentVesselDir+caseId+"/"+backwardFileName
    generateCenterline(surfaceModel,backwardCenterlineName)
#     viewCenterlineFromFile(backwardCenterlineName)
    
elif caseType == "terminal":
    # Files identifiers for terminal aneurysm cases
    parentVesselFileName = caseId+"_parentvessel.vtp" 
    firstDaughterFileName = caseId+"_dau1cl.vtp"
    secondDaughterFileName = caseId+"_dau2cl.vtp"
    
    # Centerlines for original parent vessel and branches
    print('Computing centerlines for original parent vessel and branches.\n')
    parentVesselCenterlineName = parentVesselDir+caseId+"/"+parentVesselFileName
    generateCenterline(surfaceModel,parentVesselCenterlineName)
#     viewCenterlineFromFile(parentVesselCenterlineName)
    
    # Centerlines with source the first daughter branch
    print('Computing centerlines with source the first daughter branch.\n')
    firstDauCenterlineName = parentVesselDir+caseId+"/"+firstDaughterFileName
    generateCenterline(surfaceModel,firstDauCenterlineName)
#     viewCenterlineFromFile(firstDauCenterlineName)
    
    # Centerlines with source the second daughter branch
    print('Computing centerlines with source the second 0daughter branch.\n')
    secondDauCenterlineName = parentVesselDir+caseId+"/"+secondDaughterFileName
    generateCenterline(surfaceModel,secondDauCenterlineName)
#     viewCenterlineFromFile(secondDauCenterlineName)
    
else:
    print("Aneurysm type not defined! Choices: lateral or terminal")

Computing centerlines for original parent vessel and branches.

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...Resampling
Writing VTK XML surface file.
Done.
Computing centerlines with source the first daughter branch.

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...Resampling
Writing VTK XML surface file.
Done.
Computing centerlines with source the second 0daughter branch.

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit rendere

<hr/>
## Centerlines interpolation: **patchandinterpolatecenterlines.py**

The goal of this step is to identify on the given centerlines the locations delimiting the aneurysm sac. For each couple of centerlines the *diverging point* between the line entering the aneurysm and the one along the vessel is identified and the point one maximal sphere away from the aneurysm selected as clipping point. Eventually two clipping points are extracted for lateral aneurysms and three for terminal. The clipping points are identified on the parent vessel centerlines, the portion between them eliminated and re-created interpolating the remaining tracts.

Before this step, make sure that the python script are in your path or in your current folder and make sure that all the files for id1 are in an id1 folder and all files for id2 are in an id2 folder. If the id1 and id2 folder are also in your current folder, use . as directoryPath.

In [21]:
# patchAndInterpolateCenterlines.py
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

%run $pythonBin/patchandinterpolatecenterlines.py $parentVesselDir $caseId $caseType
print('Done.')
print('Saved files: '+caseId+'_divergingpoints.vtp, '+caseId+'_clippingpoints.vtp, '+caseId+'_patchcl.vtp and '+caseId+'_interpolatedcl.vtp.')


USAGE:
      ./patchandinterpolatecenterlines.py inputfilesDirectory caseID aneurysmType

Inputfiles Directory	 /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/parentVessel/
case ID			 case11
Aneurysm Type		 terminal

Creating Patched Centerlines
Clipping Point Ids  [123, 187, 187]
Interpolating Patched Centerlines
Done.
Saved files: case11_divergingpoints.vtp, case11_clippingpoints.vtp, case11_patchcl.vtp and case11_interpolatedcl.vtp.


<hr/>
## Clipping of Voronoi Diagram: **clipvoronoidiagram.py**

This script removes the Voronoi Diagram pertaining the aneurysm sac and the tract of parent vessel centerlines underneath it, i.e. between the clipping points. The scripts needs the complete Voronoi Diagram created from the surface model (id_voronoi.vtp) and the patched centerlines created and saved within the patcheandinterpolatecenterline.py script (id_patchcl.vtp).

In [22]:
# clipVoronoiDiagram.py
# ~~~~~~~~~~~~~~~~~~~~~

%run $pythonBin/clipvoronoidiagram.py $parentVesselDir $caseId
print('Saved files: '+caseId+'_clippedvoronoi.vtp, '+caseId+'_smoothclippedvoronoi.vtp.')


USAGE:
      ./clipvoronoidiagram.py inputfilesDirectory caseID

Inputfiles Directory	 /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/parentVessel/
case ID			 case11

Clipping Voronoi Diagram
Smoothing Voronoi Diagram
from original number of points  107199 to 82575
Saved files: case11_clippedvoronoi.vtp, case11_smoothclippedvoronoi.vtp.


<hr/>
## Interpolation of Voronoi Diagram and parent vessel model reconstruction: **paralleltransportvoronoidiagram.py**

In this step the Voronoi Diagram is reconstructed in the region where it has been deleted. This operation is performed by parallel transporting Voronoi points from one end of the patched centerlines to the end(s) on the other side of the empty region following the trajectory of the interpolated centerlines and using the parallel transport normal system built on these centerlines as a guide. Once the Voronoi Diagram has been completed, the surface is recovered by means of the vmtk C++ class vtkvmtkPolyBallModeller that merges all the spheres into a surface. More in details, a 3D regular grid is constructed on the model bounding box and the position of the grid points probed with respect to the tube function constructed on the complete Voronoi Diagram (see Antiga and Steinman, IEEE Trans Med Imag 2004). The final surface is obtained as the zero level set computed on this image.

Both the id_smoothclippedvoronoi.vtp (suggested) and the id_clippedvoronoi.vtp can be used in the script. The third parameter (smoothVoronoi) taken by the script call indicates whether the smoothed Voronoi Diagram should be used for the interpolation (smoothVoronoi = 1) or not (smoothVoronoi = 0)

In [23]:
# parallelTransportVoronoiDiagram.py
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

%run  $pythonBin/paralleltransportvoronoidiagram.py $parentVesselDir $caseId 1
print('Saved files: '+caseId+'_completevoronoi.vtp, '+caseId+'_reconstructedmodel.vtp.')


USAGE:
      ./paralleltransportvoronoidiagram.py inputfilesDirectory caseID smoothVoronoi

Inputfiles Directory	 /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/parentVessel/
case ID			 case11

Interpolating cells  0 1
from id  124 to id  186 ; number of points added  185409
Interpolating cells  1 0
from id  186 to id  124 ; number of points added  131103
Interpolating cells  0 2
from id  124 to id  186 ; number of points added  185409
Interpolating cells  2 0
from id  186 to id  124 ; number of points added  150444
Reconstructing Surface from Voronoi Diagram
Saved files: case11_completevoronoi.vtp, case11_reconstructedmodel.vtp.


In [24]:
# Visualizing final reconstructed parent vessel surface
parentVesselViewer = vmtkscripts.vmtkSurfaceViewer()

parentVesselViewer.SurfaceInputFileName = parentVesselDir+caseId+"/"+caseId+"_reconstructedmodel.vtp"
parentVesselViewer.IORead()
parentVesselViewer.Execute()

Reading VTK XML surface file.
Quit renderer


In [26]:
# Raw parent vessel surface
# parentVesselRaw = parentVesselViewer.Surface #readSurface(parentVesselDir+caseId+"/"+caseId+"_reconstructedmodel.vtp")

# Smooth
parentVesselSmooth = vmtk_functions.smoothSurface(parentVesselViewer.Surface)

# Subdivide
parentVesselSubdivide = vmtk_functions.subdivideSurface(parentVesselSmooth)

In [27]:
surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

surfaceRemesh.Surface = parentVesselSubdivide
surfaceRemesh.ElementSizeMode = "edgelength"
surfaceRemesh.TargetEdgeLength = 0.1
surfaceRemesh.Execute()

# Final smoothing
parentVessel = vmtk_functions.smoothSurface(surfaceRemesh.Surface)

# View smoothed surface
vmtk_functions.viewSurface(parentVessel)

Quit renderer


In [28]:
#MAYBE IS BETTER TO PUT THE CENTERLINE CONSTRUICTION ABOVE THE REMESHING PROCEDURE TO CHECK NON MANIFOLDNESS
# computing centerlines to use with vmtksurfaceendclipper
centerlines = vmtkscripts.vmtkCenterlines()

centerlines.Surface = parentVessel
centerlines.AppendEndPoints = 1
centerlines.Execute()

# Computing centerlines Frenet system
centerlineGeometry = vmtkscripts.vmtkCenterlineGeometry()

centerlineGeometry.Centerlines = centerlines.Centerlines
# centerlineGeometry.CenterlinesOutputFileName = centerlinesDir+'centerlineGeometry.vtp' # centerlineGeometryFile
centerlineGeometry.LineSmoothing = 1
centerlineGeometry.Execute()
# centerlineGeometry.IOWrite()

# Debug
centerlineGeometry.PrintInputMembers()
centerlineGeometry.PrintOutputMembers()

# And then execute end clipper
# SOMETINES DOES NOT WORK
surfaceEndClipper = vmtkscripts.vmtkSurfaceEndClipper()

surfaceEndClipper.Surface = centerlines.Surface
# surfaceEndClipper.SurfaceInputFileName = surfacesDir+'surfaceRemeshedSmBlended.stl'
# surfaceEndClipper.IORead()

# Setup
surfaceEndClipper.CenterlineNormals = 1
surfaceEndClipper.Centerlines = centerlines.Centerlines
surfaceEndClipper.FrenetTangentArrayName = centerlineGeometry.FrenetTangentArrayName

# surfaceEndClipper.SurfaceOutputFileName = surfacesDir+surfaceEndClippedFile
surfaceEndClipper.Execute()

vmtk_functions.viewSurface(surfaceEndClipper.Surface)
# surfaceEndClipper.IOWrite()

# Debug
surfaceEndClipper.PrintInputMembers()
surfaceEndClipper.PrintOutputMembers()

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...Input vmtkcenterlinegeometry members:
    Id = 0
    Disabled = 0
    Centerlines = vtkPolyData
    CenterlinesInputFileName = 
    LengthArrayName = Length
    CurvatureArrayName = Curvature
    TorsionArrayName = Torsion
    TortuosityArrayName = Tortuosity
    FrenetTangentArrayName = FrenetTangent
    FrenetNormalArrayName = FrenetNormal
    FrenetBinormalArrayName = FrenetBinormal
    LineSmoothing = 1
    OutputSmoothedLines = 0
    NumberOfSmoothingIterations = 100
    SmoothingFactor = 0.1
    CenterlinesOutputFileName = 
Output vmtkcenterlinegeometry members:
    Id = 0
    Centerlines = vtkPolyData
    LengthArrayName = Length
    CurvatureArrayName = Curvature
    TorsionArrayName = Torsion
    

In [29]:
surfaceEndClipper.SurfaceOutputFileName = parentVesselDir+caseId+"/"+caseId+"_reconstructedmodelSmoothed.vtp"
surfaceEndClipper.IOWrite()

Writing VTK XML surface file.


In [ ]:
# And then execute end clipper
surfaceClipper = vmtkscripts.vmtkSurfaceClipper()

surfaceClipper.Surface = parentVessel
surfaceClipper.SurfaceOutputFileName = parentVesselDir+caseId+'/parentVesselClipped.vtp'
surfaceClipper.PrintOutputMembers()
surfaceClipper.Execute()
surfaceClipper.IOWrite()

In [ ]:
# Capping the surface
surfaceCapper = vmtkscripts.vmtkSurfaceCapper()

surfaceCapper.Surface = surfaceClipper.Surface
# surfaceCapper.SurfaceInputFileName = surfacesDir+'surfaceRemeshedSm.stl'
# surfaceCapper.IORead()
surfaceCapper.SurfaceOutputFileName = parentVesselDir+caseId+'parentVesselCapped.vtp'
# Capping Method = 'simple', 'centerpoint', 'smooth', 'annular', 'concaveannular'
surfaceCapper.Method = 'centerpoint'
#surfaceCapper.NumberOfRings = 8
#surfaceCapper.ConstraintFactor = 0.9
surfaceCapper.Interactive = 0

# surfaceCapper.PrintInputMembers()
# surfaceCapper.PrintOutputMembers()

surfaceCapper.Execute()
surfaceCapper.IOWrite()

# Writer capped surface to other format
# writeSurface(surfaceCapper.Surface,casePath+"surfaces/surfaceCapped.stl","ascii")

In [ ]:
patchDir = parentVesselDir+caseId+'/patches/'
parentVesselCapped = parentVesselDir+caseId+'/parentVesselCapped.vtp'

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Surface boundary inspector
# Conveting surface to mesh

print('Inspecting surface boundary and getting reference systems. \n')
surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()

# Needs to be a .vtp surface file!! not stl
surfaceToMesh.Surface = surfaceCapper.Surface #parentVesselCapped
# surfaceToMesh.SurfaceInputFileName = surfaceCapper.Surface #parentVesselCapped
# surfaceToMesh.IORead()

# surfaceToMesh.MeshOutputFileName = casePath+'meshes/surfaceCapped.vtu'
# surfaceToMesh.PrintInputMembers()
# surfaceToMesh.PrintOutputMembers()
surfaceToMesh.Execute()
# surfaceToMesh.IOWrite()

# Inspecting
surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
surfaceBoundaryInspector.CellEntityIdsArrayName = "CellEntityIds"
surfaceBoundaryInspector.ReferenceSystemsOutputFileName = parentVesselDir+caseId+'referenceSystems.dat'

# surfaceBoundaryInspector.PrintInputMembers()
# surfaceBoundaryInspector.PrintOutputMembers()

surfaceBoundaryInspector.Execute()
surfaceBoundaryInspector.IOWrite()
surfaceBoundaryInspector.OutputText('File saved: '+surfaceBoundaryInspector.ReferenceSystemsOutputFileName+'\n')
print('\n')

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Manipulation of the reference system array 
# The code below transforms the referenceSystem.dat info
# to a numpy structured array called 'capsGeometryArray'
# which contains Center, Normals, Radius and Ids infos
# of the surface caps

import numpy as np

capsGeometryList = []
# Columns to extract
# Center position, normals, radius and ids of caps 
cols = (0, 1, 2, 3, 4, 5, 6, 13)
colsType = [('Center', tuple),
            ('Normal', tuple),
            ('Radius',float),
            ('Id',int),
            ('PatchType','U10')]

# Get array from referenceSystem.dat file
arrayDatFile = np.genfromtxt(parentVesselDir+caseId+'referenceSystems.dat',skip_header=1,usecols=cols)
for row in arrayDatFile:
    # Copy formatted to list
    Center = tuple(row[0:3])
    Normal = tuple(row[3:6])
    Radius = row[6]
    Id = row[7]
    capsGeometryList.append((Center, Normal, Radius, Id,'patch'))


# Convert to array
capsGeometryArray = np.array(capsGeometryList, dtype=colsType)
print('Reference systems arrays. \n')
print(capsGeometryArray)
# print('\n')

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Last index of patches
# --> Volume is always 0
# --> Wall is always 1
wallIndex = 1
# --> Inlet index is searched below
# Number of patches (nIds) if length of 'capsGeometryArray' + 2 (volume and wall)
# Therefore, the last id is 'nIds - 1'
nIds = len(capsGeometryArray) + 2
lastIndex = nIds - 1
print('Last numbered patch index: '+str(lastIndex)+'\n')

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Getting the inlet index by searching the patch with largest area
capsArea = np.zeros(len(capsGeometryArray), dtype=[('Area',float),('Id',int)])
capsArea['Area'] = (np.pi/4)*capsGeometryArray['Radius']**2
capsArea['Id'] = capsGeometryArray['Id']

for i in range(0, len(capsArea)):
    if capsArea['Area'][i] == np.max(capsArea['Area']):
        inletIndex = capsArea['Id'][i]
        capsGeometryArray['PatchType'][i] = 'inlet'
    else:
        capsGeometryArray['PatchType'][i] = 'outlet'

print('Inlet index (supposing that is the cap with the largest section area) is: ',inletIndex)

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Surface reorientation

# It is better to reorient the final capped surface before sending to mesh in SnappyHexMesh
# The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
# and rotates and translates it conforming one of its patches to a target reference system
# for that, it uses the output of the 

print('Reorienting surface: translation and rotation.')
surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()

# Load from file
surfaceTransform.Surface = surfaceToMesh.Surface
# surfaceTransform.SurfaceInputFileName = parentVesselCapped
# surfaceTransform.IORead()

# or use the just created surfaceCapped
# surfaceTransform.Surface = surfaceCapper.Surface

# Target reference system parameters
surfaceTransform.ReferenceOrigin  = [0, 0, 0]
surfaceTransform.ReferenceNormal1 = [0, 0, -1]
surfaceTransform.ReferenceNormal2 = [0, 0, -1]

# Surface reference system
surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems
# surfaceTransform.ReferenceSystemsInputFileName = casePath+'surfaces/referenceSystems.dat'
# surfaceTransform.IORead()
surfaceTransform.ReferenceSystemId = inletIndex
surfaceTransform.ReferenceSystemsIdArrayName = 'CellEntityIds'
surfaceTransform.ReferenceSystemsNormal1ArrayName = 'BoundaryNormals'
surfaceTransform.ReferenceSystemsNormal2ArrayName = 'BoundaryNormals'

# NEEDS TO BE A VTP FILE TO SAVE ARRAY DATA!
surfaceTransform.SurfaceOutputFileName = parentVesselDir+caseId+'/parentVesselOriented.vtp'

# surfaceTransform.PrintInputMembers()
# surfaceTransform.PrintOutputMembers()

surfaceTransform.OutputText('Transforming capped surface. \n')
surfaceTransform.Execute()
surfaceTransform.IOWrite()
surfaceTransform.OutputText('File saved: '+surfaceTransform.SurfaceOutputFileName+'\n')

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Check if patch dir exists
# If it not exists, create it

print('Decomposing surface into its patches for snappy meshing.')
import os
# patchDir = casePath+'surfaces/patches/'

if not os.path.isdir(patchDir):
    print('Creating patch dir: '+patchDir)
    os.makedirs(patchDir)

# Using vmtkThreshold script to extract patches for mesh generations in snappy
print
# Outlet initial index (to increment)
outletIndex = 1

# Surface name to decompose
surfaceOriented = readSurface(parentVesselDir+caseId+'/parentVesselOriented.vtp')
print('\n')

# File names for patches
wallFileName = patchDir+'wall.stl'
inletFileName = patchDir+'inlet.stl'

for index in range(1, lastIndex + 1):
    # File name for outlets
    outletFileName = patchDir+'outlet'+str(outletIndex)+'.stl'
    print('Extracting surface with index', index)
    
    # Instantiate vmtkthreshold
    extractThreshold = vmtkscripts.vmtkThreshold()
    extractThreshold.Surface = surfaceOriented
    extractThreshold.LowThreshold = index
    extractThreshold.HighThreshold = index
    
    if index == wallIndex:
        extractThreshold.SurfaceOutputFileName = wallFileName
    elif index == inletIndex:
        extractThreshold.SurfaceOutputFileName = inletFileName
    else:
        extractThreshold.SurfaceOutputFileName = outletFileName
        outletIndex += 1
    
#     extractThreshold.PrintInputMembers()
#     extractThreshold.PrintOutputMembers()
    extractThreshold.Execute()
    
    # Writing surface
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
    extractThreshold.IOWrite()
    print('\n')